In [1]:
import math
class EquityOption:
    r=None
    def __init__(self, CallFlag, S, K, T, sigma, **kwargs):
        self.CallFlag = CallFlag
        self.S = float(S)
        self.K = float(K)
        self.T = float(T)
        self.sigma = float(sigma)
        if 'r' in kwargs.keys():
            EquityOption.r = float(kwargs.get('r'))
        if EquityOption.r==None:
            print('risk free rate hasn\'t been set')
        self.div=kwargs.get('div') if 'div' in kwargs.keys() else 0    
    
    def __str__(self):
        return '''CallFlag: {0.CallFlag!s}\n
Spot price: {0.S!s}\n
Strike price: {0.K!s}\n
Time to maturity: {0.T!s}\n
Volatility: {0.sigma!s}\n
Dividend: {0.div!s}\n
Risk free rate: {0.r!s}\n'''.format(self)
    
    def __imul__(self, stock_split):
        self.S*=stock_split
        self.K*=stock_split
        return self
    
    @staticmethod
    def d_1BS(S,K,T,sigma,div,r):
        return (math.log(S/K)+(r-div+sigma**2/2)*T)/(sigma*math.sqrt(T))

    @staticmethod
    def d_2BS(S,K,T,sigma,div,r):
        return (math.log(S/K)+(r-div-sigma**2/2)*T)/(sigma*math.sqrt(T))
    
    @staticmethod
    def normcdf(d):
        return 1/2*(1+math.erf(d/math.sqrt(2)))
    
    @staticmethod
    def normpdf(d):
        return 1/(2*math.pi)**0.5*math.exp(-d**2/2)
    
    def optionPriceBS(self,CallFlag=None, S=None, K=None, T=None, sigma=None, div=None, r=None): 
        if CallFlag is None:
            CallFlag=self.CallFlag
        if S is None:
            S=self.S
        if K is None:
            K=self.K
        if T is None:
            T=self.T
        if sigma is None:
            sigma=self.sigma
        if r is None:
            r=EquityOption.r
        if div is None:
            div=self.div
            
        d1=EquityOption.d_1BS(S,K,T,sigma,div,r)
        d2=EquityOption.d_2BS(S,K,T,sigma,div,r)
        if CallFlag=='call':
            return S*math.exp(-div*T)*EquityOption.normcdf(d1)-K*math.exp(-r*T)*EquityOption.normcdf(d2)
        elif CallFlag=='put':
            return -S*math.exp(-div*T)*EquityOption.normcdf(-d1)+K*math.exp(-r*T)*EquityOption.normcdf(-d2)

    
    def Delta(self,CallFlag=None, S=None, K=None, T=None, sigma=None, div=None, r=None): 
        if CallFlag is None:
            CallFlag=self.CallFlag
        if S is None:
            S=self.S
        if K is None:
            K=self.K
        if T is None:
            T=self.T
        if sigma is None:
            sigma=self.sigma
        if r is None:
            r=EquityOption.r
        if div is None:
            div=self.div
        
        d1=EquityOption.d_1BS(S,K,T,sigma,div,r)
        if CallFlag=='call':
            return math.exp(-div*T)* EquityOption.normcdf(d1)
        elif CallFlag=='put':
            return -math.exp(-div*T)* EquityOption.normcdf(-d1)

    def Gamma(self,CallFlag=None, S=None, K=None, T=None, sigma=None, div=None, r=None): 
        if CallFlag is None:
            CallFlag=self.CallFlag
        if S is None:
            S=self.S
        if K is None:
            K=self.K
        if T is None:
            T=self.T
        if sigma is None:
            sigma=self.sigma
        if r is None:
            r=EquityOption.r
        if div is None:
            div=self.div
        
        d1=EquityOption.d_1BS(S,K,T,sigma,div,r)  
        return math.exp(-div*T)/(S*sigma*T**0.5) * EquityOption.normpdf(d1)

    def Theta(self,CallFlag=None, S=None, K=None, T=None, sigma=None, div=None, r=None): 
        if CallFlag is None:
            CallFlag=self.CallFlag
        if S is None:
            S=self.S
        if K is None:
            K=self.K
        if T is None:
            T=self.T
        if sigma is None:
            sigma=self.sigma
        if r is None:
            r=EquityOption.r
        if div is None:
            div=self.div
        
        d1=EquityOption.d_1BS(S,K,T,sigma,div,r)
        d2=EquityOption.d_2BS(S,K,T,sigma,div,r)
        if CallFlag=='call':
            return (-(S*sigma*math.exp(-div*T)/(2*T**0.5))*EquityOption.normpdf(d1)
                      -r*K*math.exp(-r*T)*EquityOption.normcdf(d2)
                      +div*S*math.exp(-div*T)*EquityOption.normcdf(d1))
        elif CallFlag=='put':
            return (-(S*sigma*math.exp(-div*T)/(2*T**0.5))*EquityOption.normpdf(d1)
                      +r*K*math.exp(-r*T)*EquityOption.normcdf(-d2)
                      -div*S*math.exp(-div*T)*EquityOption.normcdf(-d1))
                              
    def Vega(self,CallFlag=None, S=None, K=None, T=None, sigma=None, div=None, r=None): 
        if CallFlag is None:
            CallFlag=self.CallFlag
        if S is None:
            S=self.S
        if K is None:
            K=self.K
        if T is None:
            T=self.T
        if sigma is None:
            sigma=self.sigma
        if r is None:
            r=EquityOption.r
        if div is None:
            div=self.div
            
        d1=EquityOption.d_1BS(S,K,T,sigma,div,r)
        return S*math.exp(-div*T)*T**0.5*EquityOption.normpdf(d1)
        
    def setOption(self, CallFlag=None, S=None, K=None, T=None, sigma=None, div=None):
        if CallFlag is not None:
            self.CallFlag = CallFlag
        if S is not None:
            self.S = float(S)
        if K is not None:
            self.K = float(K)
        if T is not None:
            self.T = float(T)
        if sigma is not None:
            self.sigma = float(sigma)
        if div is not None:
            self.div = float(div)

    def impliedVolNR(self,marketPrice):
        MAXITERATION=100
        init=0.5
        for i in range(MAXITERATION):
            if abs(self.optionPriceBS(sigma=init)-marketPrice)<1e-4:
                return init
            init = init - (self.optionPriceBS(sigma=init)-marketPrice)/self.Vega(sigma=init)
        print("Newton-Raphson doesn't converge")
        return -1
    
    def impliedVolBiset(self,marketPrice):
        a=0.001;b=10
        c=(a+b)/2
        while abs(self.optionPriceBS(sigma=c)-marketPrice)>1e-4:
            if (self.optionPriceBS(sigma=a)-marketPrice)*(self.optionPriceBS(sigma=c)-marketPrice)<0:
                b=c
            elif (self.optionPriceBS(sigma=b)-marketPrice)*(self.optionPriceBS(sigma=c)-marketPrice)<0:
                a=c
            c=(a+b)/2
        return c

    @classmethod
    def setRiskFree(cls,r):
        EquityOption.r=float(r)



In [2]:
b=EquityOption('call',35,40,0.3,0.2,r=0.025)
b.optionPriceBS()

0.2530863233764249

In [3]:
print(b)

CallFlag: call

Spot price: 35.0

Strike price: 40.0

Time to maturity: 0.3

Volatility: 0.2

Dividend: 0

Risk free rate: 0.025



In [4]:
b.impliedVolNR(0.5)

0.2513083554416511

In [5]:
b.impliedVolBiset(0.5)

0.25129540252685545

In [6]:
print('Delta: ',b.Delta())
print('Gamma: ',b.Gamma())
print('Theta: ',b.Theta())
print('Vega: ',b.Vega())

Delta:  0.13659810525776656
Gamma:  0.05708714908800696
Theta:  -1.5118313366723055
Vega:  4.195905457968511


In [7]:
b*=0.5
print(b)

CallFlag: call

Spot price: 17.5

Strike price: 20.0

Time to maturity: 0.3

Volatility: 0.2

Dividend: 0

Risk free rate: 0.025



In [8]:
c=EquityOption('put',50,50,0.3,0.2,r=0.025,div=0.01)
c.optionPriceBS()

2.062494322794471

In [9]:
c.optionPriceBS(sigma=0.3)

3.146134125438188

In [10]:
print('Delta: ',c.Delta())
print('Gamma: ',c.Gamma())
print('Theta: ',c.Theta())
print('Vega: ',c.Vega())

Delta:  -0.4604358963663501
Gamma:  0.07228555415237777
Theta:  -3.217388427274263
Vega:  10.84283312285666


In [11]:
print(c)

CallFlag: put

Spot price: 50.0

Strike price: 50.0

Time to maturity: 0.3

Volatility: 0.2

Dividend: 0.01

Risk free rate: 0.025



In [12]:
c*=1/3
print(c)

CallFlag: put

Spot price: 16.666666666666664

Strike price: 16.666666666666664

Time to maturity: 0.3

Volatility: 0.2

Dividend: 0.01

Risk free rate: 0.025

